In [1]:
import requests
import json

In [2]:
from web3 import Web3

# Replace with your own Alchemy (or Infura) RPC URL
alchemy_url = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"

w3 = Web3(Web3.HTTPProvider(alchemy_url))

# Check connection
print("Connected:", w3.is_connected())


Connected: True


In [4]:
''' v2 factory '''

# Step 2: Load Uniswap V2 Factory ABI from Etherscan
FACTORY_V2_ADDR = "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f"

# Replace with your Etherscan API key
ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"
url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={FACTORY_V2_ADDR}&apikey={ETHERSCAN_API_KEY}"

response = requests.get(url)
FACTORY_ABI = json.loads(response.json()['result'])


In [5]:
import requests
import json

''' v3 factory '''

UNISWAP_V3_FACTORY = "0x1F98431c8aD98523631AE4a59f267346ea31F984"
ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"

url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={UNISWAP_V3_FACTORY}&apikey={ETHERSCAN_API_KEY}"
abi = json.loads(requests.get(url).json()['result'])

factory = w3.eth.contract(address=UNISWAP_V3_FACTORY, abi=abi)


In [8]:
TRUMP = Web3.to_checksum_address("0xaf5b4a77f71dbb0097423213869151a366627750")
WETH  = Web3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")

# Uniswap V3 fee tiers
FEE_TIERS = [100, 500, 3000, 10000]  # 0.01%, 0.05%, 0.3%, 1%


In [12]:
def get_existing_pools(tokenA, tokenB):
    pools = {}
    for fee in FEE_TIERS:
        try:
            pool = factory.functions.getPool(tokenA, tokenB, fee).call()
            if int(pool, 16) != 0:
                pools[fee] = pool
        except:
            continue
    return pools

# Try both orders: TRUMP/WETH and WETH/TRUMP
pools = get_existing_pools(TRUMP, WETH)
if not pools:
    pools = get_existing_pools(WETH, TRUMP)

print("🧠 Found pools:")
for fee, addr in pools.items():
    print(f"Fee tier {fee/100000:.2%}: {addr}")


🧠 Found pools:
Fee tier 3.00%: 0x14508678D38929B04421E94d16cb7eB3B8EeD694


In [ ]:
import requests

def get_contract_creation_block(address, api_key):
    url = f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses={address}&apikey={api_key}"
    res = requests.get(url).json()
    if res["status"] == "1":
        return int(res["result"][0]["blockNumber"])
    return None

pool_address = "0x14508678D38929B04421E94d16cb7eB3B8EeD694"
creation_block = get_contract_creation_block(pool_address, ETHERSCAN_API_KEY)
print("Pool creation block:", creation_block)

timestamp = w3.eth.get_block(creation_block)["timestamp"]
print("Pool created at:", datetime.utcfromtimestamp(timestamp))


Pool creation block: 22387974
Pool created at: 2025-05-01 09:01:23


/var/folders/5s/msqhxw9x46gfs6_9m2x8qxch0000gn/T/ipykernel_98123/1181654828.py:16: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print("Pool created at:", datetime.utcfromtimestamp(timestamp))


In [ ]:
from web3 import Web3
import requests
import time
from datetime import datetime

# Setup Alchemy RPC
ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
w3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))
assert w3.is_connected(), "❌ Not connected to Ethereum"

# Constants
ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"
POOL_ADDRESS = Web3.to_checksum_address("0x14508678D38929B04421E94d16cb7eB3B8EeD694")

# Get creation block of pool
def get_creation_block(address):
    url = f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses={address}&apikey={ETHERSCAN_API_KEY}"
    res = requests.get(url).json()
    return int(res["result"][0]["blockNumber"]) if res["status"] == "1" else None

# Get logs between creation block and 5-day window (~33,000 blocks at 13s/block)
creation_block = get_creation_block(POOL_ADDRESS)
print("📦 Pool created at block:", creation_block)

# Get human timestamp
timestamp = w3.eth.get_block(creation_block)["timestamp"]
print("⏱️ Created at:", datetime.fromtimestamp(timestamp, tz=datetime.UTC))

# Estimate toBlock
approx_blocks_5days = int(5 * 24 * 60 * 60 / 13)  # ~33,000 blocks
latest_block = w3.eth.block_number
to_block = min(creation_block + approx_blocks_5days, latest_block)

# Swap topic signature
swap_topic = w3.keccak(text="Swap(address,address,int256,int256,uint160,uint128,int24)").hex()

# Fetch logs
log_filter = {
    "fromBlock": hex(creation_block),
    "toBlock": hex(to_block),
    "address": POOL_ADDRESS,
    "topics": [swap_topic]
}

try:
    logs = w3.eth.get_logs(log_filter)
    print(f"✅ Total swap logs found: {len(logs)}")
except Exception as e:
    print("❌ Failed to fetch logs:", e)


📦 Pool created at block: 22387974
⏱️ Created at: 2025-05-01 09:01:23


/var/folders/5s/msqhxw9x46gfs6_9m2x8qxch0000gn/T/ipykernel_98123/530383923.py:26: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  print("⏱️ Created at:", datetime.utcfromtimestamp(timestamp))


❌ Failed to fetch logs: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7


In [ ]:
from web3 import Web3
import requests
from datetime import datetime, timezone
import time

# Setup
ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"
POOL_ADDRESS = Web3.to_checksum_address("0x14508678D38929B04421E94d16cb7eB3B8EeD694")

w3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))
assert w3.is_connected(), "Not connected"


# Get creation block
def get_creation_block(address):
    url = f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses={address}&apikey={ETHERSCAN_API_KEY}"
    res = requests.get(url).json()
    return int(res["result"][0]["blockNumber"]) if res["status"] == "1" else None

creation_block = get_creation_block(POOL_ADDRESS)
print("📦 Pool created at block:", creation_block)

timestamp = w3.eth.get_block(creation_block)["timestamp"]
print("⏱️ Created at:", datetime.fromtimestamp(timestamp, tz=timezone.utc))


# Prepare log filter
swap_topic = w3.keccak(text="Swap(address,address,int256,int256,uint160,uint128,int24)").hex()
end_block = min(creation_block + 33000, w3.eth.block_number)

# Batch logs in chunks
def fetch_logs_in_chunks(start, end, step=1000):
    all_logs = []
    for from_block in range(start, end + 1, step):
        to_block = min(from_block + step - 1, end)
        print(f"🔍 Fetching logs from {from_block} to {to_block}...")
        try:
            logs = w3.eth.get_logs({
                "fromBlock": from_block,
                "toBlock": to_block,
                "address": POOL_ADDRESS,
                "topics": [swap_topic]
            })
            all_logs.extend(logs)
        except Exception as e:
            print(f"❌ Error from {from_block}-{to_block}: {e}")
            time.sleep(1)  # wait before retrying
    return all_logs

logs = fetch_logs_in_chunks(creation_block, end_block)
print(f"✅ Total swap logs found: {len(logs)}")


📦 Pool created at block: 22387974
⏱️ Created at: 2025-05-01 09:01:23+00:00
🔍 Fetching logs from 22387974 to 22388973...
❌ Error from 22387974-22388973: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7
🔍 Fetching logs from 22388974 to 22389973...
❌ Error from 22388974-22389973: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7
🔍 Fetching logs from 22389974 to 22390973...
❌ Error from 22389974-22390973: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7
🔍 Fetching logs from 22390974 to 22391973...
❌ Error from 22390974-22391973: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7
🔍 Fetching logs from 22391974 to 22392973...
❌ Error from 22391974-22392973: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km

KeyboardInterrupt: 

In [23]:
import requests

ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
ETHERSCAN_API_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"

pool_address = "0x14508678D38929B04421E94d16cb7eB3B8EeD694"  # TRUMP-ETH v3 pool
from_block = 22387974
to_block = from_block + 999  # keep <= 1000 blocks

# Swap event signature for Uniswap v3
swap_topic = "0xd78ad95fa46c994b6551d0da85fc275fe613ce329c1e0d7d49d85a4fdf9cdb7b"

payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "eth_getLogs",
    "params": [{
        "fromBlock": hex(from_block),
        "toBlock": hex(to_block),
        "address": pool_address,
        "topics": [swap_topic]
    }]
}

res = requests.post(alchemy_url, json=payload)
res.raise_for_status()

logs = res.json().get("result", [])
print(f"✅ Retrieved {len(logs)} logs")


HTTPError: 400 Client Error: Bad Request for url: https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7

In [24]:
import requests

response = requests.post(
  "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7",
  headers={"Content-Type": "application/json"},
  json={
    "jsonrpc": "2.0",
    "method": "eth_getLogs",
    "params": [
      {
        "fromBlock": "0x155c406",  # block 22387974
        "toBlock": "0x155c5f4",    # block 22388596 (roughly 2 hours later)
        "address": "0x14508678D38929B04421E94d16cb7eB3B8EeD694",
        "topics": [
          "0xd78ad95fa46c994b6551d0da85fc275fe613ce329c1e0d7d49d85a4fdf9cdb7b"
        ]
      }
    ],
    "id": 1
  },
)

print(response.json())


{'jsonrpc': '2.0', 'id': 1, 'result': []}


In [26]:
import requests
import time

ALCHEMY_API_KEY = "XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
POOL = "0x14508678D38929B04421E94d16cb7eB3B8EeD694"
TOPIC = "0xd78ad95fa46c994b6551d0da85fc275fe613ce329c1e0d7d49d85a4fdf9cdb7b"

start_block = 22387974
end_block = start_block + 500
latest_block = w3.eth.block_number
max_block = latest_block

found = False

while end_block <= max_block:
    print(f"🔍 Searching blocks {start_block} to {end_block}...")

    payload = {
        "jsonrpc": "2.0",
        "method": "eth_getLogs",
        "params": [{
            "fromBlock": hex(start_block),
            "toBlock": hex(end_block),
            "address": POOL,
            "topics": [TOPIC]
        }],
        "id": 1
    }

    response = requests.post(
        f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}",
        headers={"Content-Type": "application/json"},
        json=payload
    )

    result = response.json().get("result", [])

    if result:
        print(f"✅ Found {len(result)} logs at block range {start_block}-{end_block}")
        for log in result[:3]:
            print(log)
        found = True
        break

    start_block = end_block + 1
    end_block = start_block + 500
    time.sleep(0.2)  # polite delay

if not found:
    print("❌ No swap logs found in this block range.")


🔍 Searching blocks 22387974 to 22388474...
🔍 Searching blocks 22388475 to 22388975...
🔍 Searching blocks 22388976 to 22389476...
🔍 Searching blocks 22389477 to 22389977...
🔍 Searching blocks 22389978 to 22390478...
🔍 Searching blocks 22390479 to 22390979...
🔍 Searching blocks 22390980 to 22391480...
🔍 Searching blocks 22391481 to 22391981...
🔍 Searching blocks 22391982 to 22392482...
🔍 Searching blocks 22392483 to 22392983...
🔍 Searching blocks 22392984 to 22393484...
🔍 Searching blocks 22393485 to 22393985...
🔍 Searching blocks 22393986 to 22394486...
🔍 Searching blocks 22394487 to 22394987...
🔍 Searching blocks 22394988 to 22395488...
🔍 Searching blocks 22395489 to 22395989...
🔍 Searching blocks 22395990 to 22396490...
🔍 Searching blocks 22396491 to 22396991...
🔍 Searching blocks 22396992 to 22397492...
🔍 Searching blocks 22397493 to 22397993...
🔍 Searching blocks 22397994 to 22398494...
🔍 Searching blocks 22398495 to 22398995...
🔍 Searching blocks 22398996 to 22399496...
🔍 Searching

KeyboardInterrupt: 

In [ ]:
# checking for TOAI

token = w3.to_checksum_address("0xEd291c4EA45be02481E7f8359cAaDA27d5909f42")
weth  = w3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")


def get_contract_creation_block(address, api_key):
    url = f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses={address}&apikey={api_key}"
    res = requests.get(url).json()
    if res["status"] == "1":
        return int(res["result"][0]["blockNumber"])
    return None

creation_block = get_contract_creation_block(pool_address, ETHERSCAN_API_KEY)
timestamp = w3.eth.get_block(creation_block)['timestamp']
from datetime import datetime, timezone
print("🕒 Pool created at:", datetime.fromtimestamp(timestamp, tz=timezone.utc))


swap_topic = w3.keccak(text="Swap(address,address,int256,int256,uint160,uint128,int24)").hex()


# logs = w3.eth.get_logs({
#     "fromBlock": from_block,
#     "toBlock": to_block,
#     "address": pool_address,
#     "topics": [swap_topic]
# })


NameError: name 'pool_address' is not defined

In [35]:
pip install eth-abi


  Using cached eth_abi-5.2.0-py3-none-any.whl.metadata (3.8 kB)
Using cached eth_abi-5.2.0-py3-none-any.whl (28 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Imports
from web3 import Web3
import requests, json
from datetime import datetime, timezone
import pandas as pd
from eth_abi.abi import decode_abi


# ① — Setup providers and contract ABIs
ALCHEMY_URL = "https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"
ETHERSCAN_KEY = "NP263JVHJEZUFCC72UQKX93JTDZJ8WPGUW"
w3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))
assert w3.is_connected(), "Node not connected"

# ② — Uniswap V3 Factory ABI & instance
FACTORY = "0x1F98431c8aD98523631AE4a59f267346ea31F984"
url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={FACTORY}&apikey={ETHERSCAN_KEY}"
factory_abi = json.loads(requests.get(url).json()['result'])
factory = w3.eth.contract(address=FACTORY, abi=factory_abi)

# ③ — Define tokens
TOKEN = w3.to_checksum_address("0xEd291c4EA45be02481E7f8359cAaDA27d5909f42")
WETH  = w3.to_checksum_address("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")
FEES = [100, 500, 3000, 10000]

# ④ — Find the pool(s)
pools = {}
for fee in FEES:
    addr = factory.functions.getPool(TOKEN, WETH, fee).call()
    if int(addr, 16) != 0:
        pools[fee] = w3.to_checksum_address(addr)
print("Pools found:", pools)

# Choose fee—you can pick the only or the earliest created one
fee, pool = next(iter(pools.items()))
print("➡️ Using pool:", pool, "with fee tier:", fee)

# ⑤ — Get pool creation block
def get_creation_block(addr):
    r = requests.get(f"https://api.etherscan.io/api?module=contract&action=getcontractcreation&contractaddresses={addr}&apikey={ETHERSCAN_KEY}")
    j = r.json()
    return int(j["result"][0]["blockNumber"]) if j["status"] == "1" else None

creation = get_creation_block(pool)
print("Pool created at block:", creation, " time:", datetime.fromtimestamp(w3.eth.get_block(creation)['timestamp'], tz=timezone.utc))

# ⑥ — Fetch swap logs with batching
swap_topic = w3.keccak(text="Swap(address,address,int256,int256,uint160,uint128,int24)").hex()
start = creation
end = min(start + int(5*24*3600/12), w3.eth.block_number)  # ~5 days window

logs = []
step = 600  # ~2 hours per chunk
for fb in range(start, end, step):
    tb = min(fb + step - 1, end)
    resp = requests.post(ALCHEMY_URL,
        json={"jsonrpc":"2.0","id":1,"method":"eth_getLogs",
              "params":[{"fromBlock":hex(fb),"toBlock":hex(tb),"address":pool,"topics":[swap_topic]}]})
    batch = resp.json().get("result", [])
    logs += batch
print(f"Fetched {len(logs)} Swap logs")

# ⑦ — Decode swaps and build 5-min candles
records = []
for log in logs:
    values = decode_abi(["int256","int256","uint160","uint128","int24"], bytes.fromhex(log["data"][2:]))
    a0, a1 = values[0], values[1]
    ts = w3.eth.get_block(int(log["blockNumber"], 16))["timestamp"]
    price = abs(a1/a0) if a0 else None
    if price:
        records.append({"time": datetime.fromtimestamp(ts, tz=timezone.utc), "price": price})

df = pd.DataFrame(records).set_index("time")
ohlcv = df['price'].resample("5T").agg(['first','max','min','last','count'])
ohlcv.columns = ['open','high','low','close','trades']
print(ohlcv.head())


ImportError: cannot import name 'decode_abi' from 'eth_abi.abi' (/opt/anaconda3/lib/python3.12/site-packages/eth_abi/abi.py)

In [3]:
from web3 import Web3
from web3._utils.events import get_event_data

# Your Alchemy RPC
w3 = Web3(Web3.HTTPProvider("https://eth-mainnet.g.alchemy.com/v2/XtWfn0d5ooMWIVs5AQMxOAb474Km-hs7"))

# ABI for Uniswap V3 Swap event
swap_event_abi = {
    "anonymous": False,
    "inputs": [
        {"indexed": True, "internalType": "address", "name": "sender", "type": "address"},
        {"indexed": True, "internalType": "address", "name": "recipient", "type": "address"},
        {"indexed": False, "internalType": "int256", "name": "amount0", "type": "int256"},
        {"indexed": False, "internalType": "int256", "name": "amount1", "type": "int256"},
        {"indexed": False, "internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
        {"indexed": False, "internalType": "uint128", "name": "liquidity", "type": "uint128"},
        {"indexed": False, "internalType": "int24", "name": "tick", "type": "int24"},
    ],
    "name": "Swap",
    "type": "event"
}

# Example raw log (replace this with what you get from eth_getLogs)
log = {
    "address": "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8",
    "topics": [
        "0xd78ad95fa46c994b6551d0da85fc275fe613ce329c1e0d7d49d85a4fdf9cdb7b",
        "0x000000000000000000000000a16081f360e3847006db660bae1c6d1b2e17ec2a",
        "0x000000000000000000000000f39fd6e51aad88f6f4ce6ab8827279cfffb92266"
    ],
    "data": "0x" +
        "fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffd8f60" +  # amount0
        "0000000000000000000000000000000000000000000000000000000000005c28" +  # amount1
        "0000000000000000000000000000000000000000000000000000000001f4a1d3" +  # sqrtPriceX96
        "00000000000000000000000000000000000000000000000000000000000f4240" +  # liquidity
        "fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffef0",  # tick
    "blockNumber": "0x123456",
    "transactionHash": "0xabc...",
    "transactionIndex": "0x0",
    "blockHash": "0xabc...",
    "logIndex": "0x0",
    "removed": False
}

decoded = get_event_data(w3.codec, swap_event_abi, log)
print(decoded['args'])


MismatchedABI: The event signature did not match the provided ABI